In [3]:
import importlib
import requests
from PIL import Image
import imghdr
import os 
import cv2
from io import BytesIO
import json
import numpy as np

# отправка запросов с видео файлом

In [5]:
#файл, который отправляем
img = r'D:\projects\ero_video\dataset\insta\011.mp4' #без текста
img = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\5b3012c99c38ea7955b363b9f1fb15bb805827a1.mp4' #видео с текстом
img = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\7eef700e450d03ea7083a307636179958348f6bf.mp4' #видео с текстом ВЫДАЕТ ОШИБКУ

img = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\12e8647a0c07e16045a912a8953297061e1e804e.mp4' 
img = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\a7ee7b99b46830da26d322e9f75e4a3c9f04aab0.mp4' 



url = "http://127.0.0.1:8000/vd"  #ЮРЛ - другой. Он отличается от картинки
file = {"video": open(img, "rb")}  #открываем файл как бинарный

response = requests.post(url, files=file)

print(response.headers)
if response.status_code == 200:
   
    result = response.json()
    print( result)

else:
    print(f"Error: {response.status_code} - {response.text}")



#ОПИСАНИЕ СТРУКТУРЫ ОТВЕТА
#json структура. 
#need_moderation
    #1 - нужна модерация
    #0 - модераци не нужна, текстов не найдено

# Если need_moderation = 1, то появится еще один ключ timeslots
# В нем в списке хранятся словари {'timesslot': [sN, sF], 'frame_data': [список целых чисел]}
# timesslot - начало и окончание кадров с текстом в видео. 
# frame_data - первый кадр, где начинается текст
# Таких словарей может быть несколько в зависимости от того, сколько найдено таймслотов. 

# Как из framme_data получать изображение:
# jpeg_bytes_list = record['frame_data'] #в обзем виде это массив чисел, который сейчас будем перепаковывать в jpg
# jpeg_bytes_array = np.array(jpeg_bytes_list, dtype=np.uint8)
# image = cv2.imdecode(jpeg_bytes_array, flags=cv2.IMREAD_COLOR)

# image - будет содержать картинку. Далее с ним можно делать всё, что хотим: сохранить, показать,  и т.д... далее есть пример


{'date': 'Mon, 26 Aug 2024 06:31:52 GMT', 'server': 'uvicorn', 'content-length': '368522', 'content-type': 'application/json'}
{'need_moderation': 1, 'timeslots': [{'fps': 30.0, 'timesslotF': [0, 302], 'timesslot': [0, 11], 'frame_data': [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 255, 219, 0, 67, 0, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 4, 3, 2, 2, 2, 2, 5, 4, 4, 3, 4, 6, 5, 6, 6, 6, 5, 6, 6, 6, 7, 9, 8, 6, 7, 9, 7, 6, 6, 8, 11, 8, 9, 10, 10, 10, 10, 10, 6, 8, 11, 12, 11, 10, 12, 9, 10, 10, 10, 255, 219, 0, 67, 1, 2, 2, 2, 2, 2, 2, 5, 3, 3, 5, 10, 7, 6, 7, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 255, 192, 0, 17, 8, 2, 128, 1, 224, 3, 1, 34, 0, 2, 17, 1, 3, 17, 1, 255, 196, 0, 31, 0, 0, 1, 5, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 255, 196, 0, 181, 16, 0,

# ПОЛНЫЙ ПРИМЕР

In [ ]:
#перебираем все файлы, отправляем их на модерацию, получам и обрабатываем ответ, сохраняем полученные кадры с тектом

In [14]:
#функция получает ответ API, и достает из jsona все jpg и таймсслоты
def unpack_jpg_from_json(rez,item=''):
    if rez['need_moderation']:
        print('нужна модерация')
        #достаем фото
        for record in rez['timeslots']:
            print(record['timesslot']) #это наш таймслот. их в ответе может быть от 1 до много
            jpeg_bytes_list = record['frame_data'] #в обзем виде это массив чисел, который сейчас будем перепаковывать в jpg
            jpeg_bytes_array = np.array(jpeg_bytes_list, dtype=np.uint8)
            image = cv2.imdecode(jpeg_bytes_array, flags=cv2.IMREAD_COLOR)
            #сохраняем сюда
            key_frame = record['timesslot'][0] #ключевой кадр
            cv2.imwrite(f'Y:\decoded_image {item} {key_frame}.jpg', image)
    else:
        print('Модерация пройдена')





In [15]:
url = "http://127.0.0.1:8000/vd"  #ЮРЛ - другой. 
    
def get_moderation_video_file(video_file,item):
    file = {"video": open(video_file, "rb")}  #открываем файл как бинарный

    response = requests.post(url, files=file)
    if response.status_code == 200:
        result = response.json()
        #print(result)
        unpack_jpg_from_json(result,item)
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [16]:
#Тестируем. Перебираем все файлы полученные
import os

directory_path = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом'
# directory_path = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\видео без текста'
#directory_path =  r'D:\projects\ero_txt\_VIDEO\dataset\test 1000'

files_and_dirs = os.listdir(directory_path)

for item in files_and_dirs:
    work_file = os.path.join(directory_path, item)
    print(work_file)
    get_moderation_video_file(work_file,item)

D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\10c077af2ed907fb9845c8abf5bf666338815030.mp4
нужна модерация
[0, 25]
[75, 205]
[240, 325]
D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\11e1cbb141b986aa80fc984378d41d35b87a1500.mp4
нужна модерация
[28, 60]
[97, 113]
D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\12e8647a0c07e16045a912a8953297061e1e804e.mp4
нужна модерация
[0, 73]
[122, 292]
[516, 534]
D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\17fbcb5b61294b8ba39061d3ec2e5a745a96e34d.mp4
нужна модерация
[0, 147]
[194, 462]
D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\1abb74083625da9382dd22b9c7731f4b8b4a1fc6.mp4
нужна модерация
[122, 135]
D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\391f5ceaa7687acc57f6120ba747fb108970e07b.mp4
нужна модерация
[0, 11]
D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\49233673a66de81a06069099d4e3c6b424a4c7e4.mp4
нужна модерация
[0, 37]
D:\projects\ero_txt\_VIDEO\dataset\

# отправка запросов с изображением файлом

In [26]:
img  = r'D:\projects\ero_txt\txt_girls\data_set\inp_2023-12-03 19-09-00.jpg' #пример с текстом
#img  = r'D:\projects\ero_txt\Оружие\2024-05-20_12-09-55.png'

url = "http://localhost:8000/ps"
file = {"image": open(img, "rb")}  #открываем файл как бинарный


response = requests.post(url, files=file)

#print(response.headers)
if response.status_code == 200:
    result = response.json()
    print( result)

#json структура. 
#need_moderation
    #1 - нужна модерация
    #0 - модераци не нужна, текстов не найдено

{'need_moderation': [1]}
